# Python Lab — Module 00, Unit 02: Dot Products & Cross Products

> **Threat Surfaces: Multivariable Calculus for AI Security**  
> fischer³ Education | Module 00 | Unit 02
>
> **Estimated time**: 20–25 minutes  
> **Prerequisite**: Complete `notes.md` and attempt `exercises.tex` before running this lab

---

## What This Lab Does

The exercises built computational fluency with dot products, projections, and cross products. This lab adds three things the exercises cannot:

1. **Geometric visualization** of projection and orthogonal decomposition in $\mathbb{R}^2$
2. **3D visualization** of the cross product as a perpendicular vector
3. **OLS regression geometry** — the projection interpretation of least squares, visualized in $\mathbb{R}^2$ and confirmed computationally

The OLS section is the most important. By the end you will have a concrete visual image of what it means for residuals to be orthogonal to the fitted values — an image you can carry forward through the rest of the course.

## Lab Objectives

- [ ] Verify Exercise 1–2 computations symbolically with SymPy
- [ ] Visualize vector projection and orthogonal decomposition in $\mathbb{R}^2$
- [ ] Visualize the cross product as a perpendicular vector in $\mathbb{R}^3$
- [ ] Build and visualize the OLS projection geometry from Exercise 3
- [ ] Plot the angle between two vectors and see $\cos\theta$ geometrically

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from mpl_toolkits.mplot3d import Axes3D
import sympy as sp
from sympy import Matrix, sqrt, acos, pi, simplify, Rational, cos, latex
from sympy import symbols

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({
    'figure.figsize': (9, 6),
    'font.size': 11,
    'axes.titlesize': 13,
    'axes.labelsize': 11,
    'lines.linewidth': 2,
    'figure.dpi': 120
})

TS_BLUE  = '#1e64b4'
TS_AMBER = '#c87814'
TS_GREEN = '#1e8c50'
TS_GRAY  = '#64646e'
TS_RED   = '#b41e1e'
TS_LIGHT = '#f5f7fa'

def draw_vector(ax, origin, vec, color, label='', lw=2.0, ls='-', alpha=1.0, offset=(0.05, 0.05)):
    """Draw a 2D vector as an arrow from origin with an optional label."""
    ax.annotate('', xy=origin + vec, xytext=origin,
                arrowprops=dict(arrowstyle='->', color=color, lw=lw, linestyle=ls, alpha=alpha))
    if label:
        mid = origin + vec * 0.6
        ax.text(mid[0] + offset[0], mid[1] + offset[1], label,
                color=color, fontsize=12, fontweight='bold')

print('Imports complete.')

---

## Section 1 — Symbolic Verification

We verify Exercises 1 and 2 using exact SymPy arithmetic before moving to visualization.

In [ ]:
# --- Exercise 1 vectors ---
u = Matrix([3, -2, 1])
v = Matrix([1,  4, 2])
w = Matrix([2,  3, -8])

uv = u.dot(v)
uw = u.dot(w)
vw = v.dot(w)

print('Exercise 1 dot products:')
print(f'  u · v = {uv}')
print(f'  u · w = {uw}')
print(f'  v · w = {vw}')
print()

# Orthogonality check
pairs = [('u,v', uv), ('u,w', uw), ('v,w', vw)]
for name, dp in pairs:
    status = 'ORTHOGONAL ⊥' if dp == 0 else f'not orthogonal (dot product = {dp})'
    print(f'  {name}: {status}')

print()
# Norm verification
norm_u_sq_direct = u.dot(u)
norm_u_sq_formula = sum(x**2 for x in u)
print(f'||u||² via u·u = {norm_u_sq_direct}')
print(f'||u||² via sum of squares = {norm_u_sq_formula}')
print(f'Equal: {norm_u_sq_direct == norm_u_sq_formula}')

In [ ]:
# --- Exercise 2 vectors ---
p = Matrix([4, 1])
q = Matrix([1, 3])

dot_pq   = p.dot(q)
norm_p   = sqrt(p.dot(p))
norm_q   = sqrt(q.dot(q))
cos_theta = dot_pq / (norm_p * norm_q)
theta_rad = acos(cos_theta)
theta_deg = theta_rad * 180 / pi

print('Exercise 2:')
print(f'  p · q     = {dot_pq}')
print(f'  ||p||     = {norm_p} ≈ {float(norm_p):.4f}')
print(f'  ||q||     = {norm_q} ≈ {float(norm_q):.4f}')
print(f'  cos θ     = {simplify(cos_theta)} ≈ {float(cos_theta):.4f}')
print(f'  θ (exact) = arccos({simplify(cos_theta)})')
print(f'  θ (deg)   ≈ {float(theta_deg):.2f}°')

# Projection of p onto q
proj_p_on_q = (dot_pq / q.dot(q)) * q
p_perp = p - proj_p_on_q

print(f'\n  proj_q(p) = {proj_p_on_q.T}')
print(f'  p_⊥       = {p_perp.T}')
print(f'  p_⊥ · q   = {p_perp.dot(q)}  (should be 0)')

# Pythagorean identity
lhs = p.dot(p)
rhs = proj_p_on_q.dot(proj_p_on_q) + p_perp.dot(p_perp)
print(f'\n  ||p||²                         = {lhs}')
print(f'  ||proj||² + ||p_⊥||²           = {simplify(rhs)}')
print(f'  Pythagorean identity holds: {simplify(lhs - rhs) == 0}')

---

## Section 2 — Visualizing Projection & Orthogonal Decomposition

We now plot the projection geometry from Exercise 2 in $\mathbb{R}^2$. This is the core geometric picture of the entire unit — build a strong mental image of it here, because it recurs in the regression section below and throughout the course.

In [ ]:
# NumPy versions for plotting
p_np = np.array([4, 1], dtype=float)
q_np = np.array([1, 3], dtype=float)

# Compute projection numerically
proj_scalar = np.dot(p_np, q_np) / np.dot(q_np, q_np)
proj_vec    = proj_scalar * q_np
p_perp_np   = p_np - proj_vec

fig, axes = plt.subplots(1, 2, figsize=(14, 7))

# --- Left panel: The full decomposition ---
ax = axes[0]
origin = np.array([0.0, 0.0])

draw_vector(ax, origin, p_np,      TS_BLUE,  r'$\mathbf{p}$', lw=2.5)
draw_vector(ax, origin, q_np,      TS_GREEN, r'$\mathbf{q}$', lw=2.5)
draw_vector(ax, origin, proj_vec,  TS_AMBER, r'$\mathrm{proj}_{\mathbf{q}}\mathbf{p}$', lw=2.5)
draw_vector(ax, proj_vec, p_perp_np, TS_RED, r'$\mathbf{p}_\perp$', lw=2.0, ls='--')

# Right angle marker at the projection point
box_size = 0.15
perp_dir = p_perp_np / np.linalg.norm(p_perp_np) * box_size
q_dir    = q_np / np.linalg.norm(q_np) * box_size
corner1  = proj_vec + perp_dir
corner2  = proj_vec + perp_dir + q_dir
corner3  = proj_vec + q_dir
box_pts  = np.array([proj_vec, corner1, corner2, corner3, proj_vec])
ax.plot(box_pts[:,0], box_pts[:,1], color=TS_GRAY, lw=1.0)

# Extend q line for context
q_line = q_np / np.linalg.norm(q_np)
ax.plot([-q_line[0]*0.5, q_line[0]*5.5],
        [-q_line[1]*0.5, q_line[1]*5.5],
        color=TS_GREEN, lw=0.8, linestyle=':', alpha=0.5)

ax.set_xlim(-0.5, 5.5)
ax.set_ylim(-0.5, 5.5)
ax.set_aspect('equal')
ax.axhline(0, color=TS_GRAY, lw=0.6)
ax.axvline(0, color=TS_GRAY, lw=0.6)
ax.set_xlabel('$x_1$', fontsize=13)
ax.set_ylabel('$x_2$', fontsize=13)
ax.set_title('Projection & Orthogonal Decomposition\n'
             r'$\mathbf{p} = \mathrm{proj}_{\mathbf{q}}\mathbf{p} + \mathbf{p}_\perp$',
             fontweight='bold', color=TS_BLUE)

# --- Right panel: Angle visualization ---
ax2 = axes[1]
theta_val = np.arccos(np.dot(p_np, q_np) / (np.linalg.norm(p_np) * np.linalg.norm(q_np)))

# Draw unit vectors to show angle
p_hat = p_np / np.linalg.norm(p_np)
q_hat = q_np / np.linalg.norm(q_np)

draw_vector(ax2, origin, p_np, TS_BLUE,  r'$\mathbf{p}$', lw=2.5)
draw_vector(ax2, origin, q_np, TS_GREEN, r'$\mathbf{q}$', lw=2.5)

# Arc showing angle
arc_r = 0.9
p_angle = np.arctan2(p_np[1], p_np[0])
q_angle = np.arctan2(q_np[1], q_np[0])
arc_angles = np.linspace(p_angle, q_angle, 100)
ax2.plot(arc_r * np.cos(arc_angles), arc_r * np.sin(arc_angles),
         color=TS_AMBER, lw=2.0)
mid_arc = (p_angle + q_angle) / 2
ax2.text(arc_r * 1.15 * np.cos(mid_arc), arc_r * 1.15 * np.sin(mid_arc),
         f'θ ≈ {np.degrees(theta_val):.1f}°', color=TS_AMBER,
         fontsize=11, ha='center')

# Cos theta annotation
cos_val = np.dot(p_np, q_np) / (np.linalg.norm(p_np) * np.linalg.norm(q_np))
ax2.text(0.3, -0.3, f'cos θ = p̂·q̂ ≈ {cos_val:.3f}',
         fontsize=10, color=TS_GRAY,
         bbox=dict(boxstyle='round,pad=0.3', facecolor=TS_LIGHT, edgecolor=TS_GRAY))

ax2.set_xlim(-0.5, 5.5)
ax2.set_ylim(-0.5, 4.5)
ax2.set_aspect('equal')
ax2.axhline(0, color=TS_GRAY, lw=0.6)
ax2.axvline(0, color=TS_GRAY, lw=0.6)
ax2.set_xlabel('$x_1$', fontsize=13)
ax2.set_ylabel('$x_2$', fontsize=13)
ax2.set_title('Angle Between Vectors\n'
              r'$\cos\theta = \hat{\mathbf{p}} \cdot \hat{\mathbf{q}}$',
              fontweight='bold', color=TS_BLUE)

plt.suptitle('Module 00, Unit 02: Projection & Angle Geometry',
             fontsize=13, fontweight='bold', color=TS_GRAY, y=1.01)
plt.tight_layout()
plt.savefig('../../../assets/figures/m00-u02-projection.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'proj_q(p) = {proj_vec}')
print(f'p_perp    = {p_perp_np}')
print(f'p_perp · q = {np.dot(p_perp_np, q_np):.10f}  (≈ 0)')

### What Do You See?

1. **Left panel**: $\mathbf{p}$ is decomposed into two parts — one along $\mathbf{q}$ (the projection, amber) and one perpendicular to $\mathbf{q}$ (the orthogonal component, red dashed). The right-angle marker at the projection point confirms they are exactly perpendicular.

2. **Right panel**: The angle $\theta$ is the opening between the two vectors. Its cosine is precisely $\hat{\mathbf{p}} \cdot \hat{\mathbf{q}}$ — the dot product of the normalized vectors.

**Analysis.** Hold this picture in mind — it is the geometric heart of the course. The decomposition $\mathbf{p} = \text{proj}_{\mathbf{q}}\mathbf{p} + \mathbf{p}_\perp$ extends directly to $\mathbb{R}^n$: in regression, $\mathbf{y}$ plays the role of $\mathbf{p}$, the column space of $\mathbf{X}$ plays the role of $\mathbf{q}$, and the residual $\mathbf{e}$ plays the role of $\mathbf{p}_\perp$.

---

## Section 3 — Cross Product in 3D

We visualize the cross product from Exercise 4 as a vector perpendicular to both $\mathbf{a}$ and $\mathbf{b}$, and show the parallelogram whose area it encodes.

In [ ]:
# --- Vectors from Exercise 4 ---
a_np = np.array([2,  1, -1], dtype=float)
b_np = np.array([-1, 3,  2], dtype=float)
axb  = np.cross(a_np, b_np)

print(f'a       = {a_np}')
print(f'b       = {b_np}')
print(f'a × b   = {axb}')
print(f'||a × b|| = {np.linalg.norm(axb):.4f}')
print(f'Parallelogram area = {np.linalg.norm(axb):.4f}')
print(f'(a × b) · a = {np.dot(axb, a_np):.6f}  (should be 0)')
print(f'(a × b) · b = {np.dot(axb, b_np):.6f}  (should be 0)')

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

origin = np.array([0, 0, 0])

# Scale cross product for visual clarity
scale_axb = axb / np.linalg.norm(axb) * 2.5

def draw_vector_3d(ax, origin, vec, color, label, lw=2):
    ax.quiver(*origin, *vec, color=color, lw=lw,
              arrow_length_ratio=0.12)
    tip = origin + vec
    ax.text(tip[0], tip[1], tip[2] + 0.15, label,
            color=color, fontsize=12, fontweight='bold')

draw_vector_3d(ax, origin, a_np,      TS_BLUE,  r'$\mathbf{a}$')
draw_vector_3d(ax, origin, b_np,      TS_GREEN, r'$\mathbf{b}$')
draw_vector_3d(ax, origin, scale_axb, TS_RED,   r'$\mathbf{a} \times \mathbf{b}$')

# Draw parallelogram
para = np.array([origin,
                 origin + a_np,
                 origin + a_np + b_np,
                 origin + b_np,
                 origin])
ax.plot(para[:,0], para[:,1], para[:,2],
        color=TS_AMBER, lw=1.5, linestyle='--', alpha=0.7, label='Parallelogram')

# Fill parallelogram
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
verts = [[origin, origin + a_np, origin + a_np + b_np, origin + b_np]]
poly  = Poly3DCollection(verts, alpha=0.15, facecolor=TS_AMBER, edgecolor=TS_AMBER)
ax.add_collection3d(poly)

ax.set_xlabel('$x_1$', fontsize=11)
ax.set_ylabel('$x_2$', fontsize=11)
ax.set_zlabel('$x_3$', fontsize=11)
ax.set_title('Cross Product: $\\mathbf{a} \\times \\mathbf{b}$ is Perpendicular to Both\n'
             f'Parallelogram area = $\\|\\mathbf{{a}} \\times \\mathbf{{b}}\\|$ ≈ {np.linalg.norm(axb):.3f}',
             fontweight='bold', color=TS_BLUE)

ax.text2D(0.02, 0.02,
          f'a × b = ({axb[0]:.0f}, {axb[1]:.0f}, {axb[2]:.0f})ᵀ\n'
          f'Verify: (a×b)·a = {np.dot(axb, a_np):.1f}, (a×b)·b = {np.dot(axb, b_np):.1f}',
          transform=ax.transAxes, fontsize=9, color=TS_GRAY,
          bbox=dict(boxstyle='round', facecolor=TS_LIGHT, edgecolor=TS_GRAY))

plt.tight_layout()
plt.savefig('../../../assets/figures/m00-u02-cross-product.png', dpi=150, bbox_inches='tight')
plt.show()

### What Do You See?

1. The red vector ($\mathbf{a} \times \mathbf{b}$) points straight out of the plane of $\mathbf{a}$ and $\mathbf{b}$ — visually perpendicular to the shaded parallelogram.
2. The parallelogram spanned by $\mathbf{a}$ and $\mathbf{b}$ has area equal to $\|\mathbf{a} \times \mathbf{b}\|$.

**Analysis.** The cross product encodes two pieces of information simultaneously: the normal direction to a plane (used in Module 07 for surface integrals) and the area of the parallelogram. When $\mathbf{a}$ and $\mathbf{b}$ are parallel, $\sin\theta = 0$ and the parallelogram collapses — which is why $\mathbf{a} \times \mathbf{a} = \mathbf{0}$.

---

## Section 4 — Statistical Bridge: OLS Projection Geometry

This is the most important section of the lab. We visualize the OLS projection from Exercise 3 — confirming the orthogonality of residuals and seeing the Pythagorean decomposition geometrically.

### Setup

We have $\mathbf{y} = (2, 4, 5, 3)^\top$ (responses) and $\mathbf{x} = (1, 2, 3, 2)^\top$ (predictor), both in $\mathbb{R}^4$. The fitted values are $\hat{\mathbf{y}} = \hat{\beta}\,\mathbf{x}$ where $\hat{\beta} = (\mathbf{y} \cdot \mathbf{x}) / (\mathbf{x} \cdot \mathbf{x})$.

In [ ]:
# --- OLS computation (Exercise 3) ---
y = np.array([2, 4, 5, 3], dtype=float)
x = np.array([1, 2, 3, 2], dtype=float)

# Projection
beta_hat  = np.dot(y, x) / np.dot(x, x)
y_hat     = beta_hat * x
residuals = y - y_hat

print('OLS via projection formula:')
print(f'  y · x    = {np.dot(y, x)}')
print(f'  x · x    = {np.dot(x, x)}')
print(f'  β̂        = {beta_hat:.4f}')
print(f'  ŷ        = {y_hat}')
print(f'  e = y−ŷ  = {residuals}')
print()
print(f'Orthogonality check: e · x = {np.dot(residuals, x):.10f}  (should be 0)')
print()

# Pythagorean decomposition
ss_total   = np.dot(y, y)
ss_fitted  = np.dot(y_hat, y_hat)
ss_resid   = np.dot(residuals, residuals)
print('Pythagorean decomposition:')
print(f'  ||y||²   = {ss_total:.4f}')
print(f'  ||ŷ||²   = {ss_fitted:.4f}')
print(f'  ||e||²   = {ss_resid:.4f}')
print(f'  ||ŷ||² + ||e||² = {ss_fitted + ss_resid:.4f}  (should equal ||y||² = {ss_total:.4f})')

In [ ]:
# --- Traditional regression scatter plot ---
# Plot y vs x, fitted line, and residuals as vertical segments

x_obs = np.array([1, 2, 3, 2], dtype=float)  # predictor values
y_obs = np.array([2, 4, 5, 3], dtype=float)  # observed responses
y_fit = beta_hat * x_obs                      # fitted values

x_line = np.linspace(0.5, 3.5, 100)
y_line = beta_hat * x_line

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Left: scatter with regression line and residuals
ax1 = axes[0]
ax1.plot(x_line, y_line, color=TS_BLUE, lw=2, label=f'Fitted: $\\hat{{y}} = {beta_hat:.2f}x$')
ax1.scatter(x_obs, y_obs, color=TS_BLUE, s=80, zorder=5, label='Observed $y_i$')
ax1.scatter(x_obs, y_fit, color=TS_AMBER, s=60, zorder=5, marker='s', label='Fitted $\\hat{y}_i$')

# Draw residuals as vertical segments
for xi, yi, yhi in zip(x_obs, y_obs, y_fit):
    ax1.plot([xi, xi], [yi, yhi],
             color=TS_RED, lw=1.5, linestyle='--', alpha=0.8)
    ax1.annotate(f'{yi - yhi:+.2f}',
                 xy=(xi + 0.05, (yi + yhi) / 2),
                 fontsize=9, color=TS_RED)

ax1.set_xlabel('$x$ (predictor)', fontsize=12)
ax1.set_ylabel('$y$ (response)', fontsize=12)
ax1.set_title('OLS Regression: Observed, Fitted, Residuals',
              fontweight='bold', color=TS_BLUE)
ax1.legend(fontsize=10)
ax1.text(0.55, 0.08,
         f'RSS = $\\|\\mathbf{{e}}\\|^2$ = {ss_resid:.3f}',
         transform=ax1.transAxes, fontsize=10, color=TS_RED,
         bbox=dict(boxstyle='round', facecolor=TS_LIGHT, edgecolor=TS_RED))

# Right: Pythagorean bar chart showing decomposition
ax2 = axes[1]
categories = ['$\\|\\mathbf{y}\\|^2$\n(Total)', 
              '$\\|\\hat{\\mathbf{y}}\\|^2$\n(Fitted)', 
              '$\\|\\mathbf{e}\\|^2$\n(Residual)']
values = [ss_total, ss_fitted, ss_resid]
colors = [TS_BLUE, TS_AMBER, TS_RED]

bars = ax2.bar(categories, values, color=colors, width=0.5, alpha=0.8, edgecolor='white')
for bar, val in zip(bars, values):
    ax2.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.3,
             f'{val:.3f}', ha='center', va='bottom', fontweight='bold', fontsize=11)

ax2.set_ylabel('Sum of squares', fontsize=12)
ax2.set_title(f'Pythagorean Decomposition\n'
              f'$\\|\\mathbf{{y}}\\|^2 = \\|\\hat{{\\mathbf{{y}}}}\\|^2 + \\|\\mathbf{{e}}\\|^2$',
              fontweight='bold', color=TS_BLUE)
ax2.text(0.5, 0.85,
         f'{ss_fitted:.3f} + {ss_resid:.3f} = {ss_fitted+ss_resid:.3f}\n≈ {ss_total:.3f} ✓',
         transform=ax2.transAxes, fontsize=10, ha='center', color=TS_GRAY,
         bbox=dict(boxstyle='round', facecolor=TS_LIGHT, edgecolor=TS_GRAY))

plt.suptitle('Module 00, Unit 02 — Statistical Bridge: OLS as Projection',
             fontsize=13, fontweight='bold', color=TS_GRAY)
plt.tight_layout()
plt.savefig('../../../assets/figures/m00-u02-ols-projection.png', dpi=150, bbox_inches='tight')
plt.show()

### What Do You See?

1. **Left panel**: The red dashed segments are the residuals — the vertical distance between each observed point and the fitted line. Their lengths squared sum to the RSS.

2. **Right panel**: The Pythagorean decomposition holds exactly. The total sum of squares ($\|\mathbf{y}\|^2$) splits perfectly into fitted and residual components.

**Analysis.** This is the geometric content of $R^2$. The ratio $\|\hat{\mathbf{y}}\|^2 / \|\mathbf{y}\|^2$ measures what fraction of the total variation is explained by the fit — but note this is the *uncentered* version. The standard $R^2$ centers by subtracting the mean first. The Pythagorean structure is the same in both cases.

The orthogonality condition $\mathbf{e} \cdot \mathbf{x} = 0$ is not a coincidence or a nice side effect of OLS — it *defines* the OLS estimate. We chose $\hat{\beta}$ to be the value that makes the residuals orthogonal to the predictor. That is what least squares means geometrically.

---

## Extension Challenge (Optional)

**Challenge.** The projection formula we used, $\hat{\beta} = (\mathbf{y} \cdot \mathbf{x}) / (\mathbf{x} \cdot \mathbf{x})$, works for a single predictor through the origin (no intercept). The full OLS formula with an intercept is $\hat{\boldsymbol{\beta}} = (\mathbf{X}^\top\mathbf{X})^{-1}\mathbf{X}^\top\mathbf{y}$ where $\mathbf{X}$ includes a column of ones.

Using NumPy, add a column of ones to the predictor matrix, apply the full OLS formula, and compare the fit to the no-intercept version from this lab. How do the residuals and $\|\mathbf{e}\|^2$ change? What does the orthogonality condition become when there are two predictors (the ones column and $\mathbf{x}$)?

In [ ]:
# Extension workspace — full OLS with intercept
# Hint: np.linalg.lstsq or the normal equations directly


---

## Lab Summary

| What we did | Key result |
|---|---|
| Verified Exercises 1–2 symbolically | All dot products, angles, projections confirmed |
| Visualized projection + orthogonal decomposition | $\mathbf{p} = \text{proj}_{\mathbf{q}}\mathbf{p} + \mathbf{p}_\perp$, right angle confirmed visually |
| Visualized cross product in 3D | $\mathbf{a} \times \mathbf{b}$ perpendicular to both; parallelogram area visible |
| Built OLS projection from scratch | $\hat{\boldsymbol{\beta}}$ is a dot product ratio; residuals orthogonal to predictor |
| Plotted Pythagorean decomposition | $\|\mathbf{y}\|^2 = \|\hat{\mathbf{y}}\|^2 + \|\mathbf{e}\|^2$ holds exactly |

## Reflection

1. **The OLS formula $\hat{\beta} = (\mathbf{y} \cdot \mathbf{x})/(\mathbf{x} \cdot \mathbf{x})$ is a dot product ratio. What does the numerator measure, and what does the denominator normalize by?**  
   *Your answer here*

2. **We confirmed $\mathbf{e} \cdot \mathbf{x} = 0$ numerically. Can you sketch why this must be true analytically — what condition on $\hat{\beta}$ would force the residuals to be orthogonal to $\mathbf{x}$?**  
   *Your answer here*

3. **We work in $\mathbb{R}^4$ in this exercise (four observations). What would change geometrically if we had 100 observations instead?**  
   *Your answer here*

---

**Next sub-unit**: Module 00, Unit 03 — Single-Variable Review & Notation Setup  
**Solutions**: `threat-surfaces-solutions` repository, `module-00-orientation/unit-02-dot-and-cross-products/`

---
*Python Lab | Module 00, Unit 02 — Threat Surfaces, fischer³ Education*